In [1]:
from datasets.real_datasets import act_preprocesser

In [2]:
X, y = act_preprocesser("../attic/data/ATC/atc_20121024_20121205.csv")

In [3]:
import numpy as np

dataset_size = X.shape[0]
seed = 42
test_percentage = 0.1
random_number_generator = np.random.default_rng(seed=seed)
train_size, test_size = dataset_size - int(dataset_size * test_percentage), int(dataset_size * test_percentage)

all_indexes = np.arange(0, dataset_size, 1)
train_indexes = np.random.choice(all_indexes, train_size, replace=False)
test_mask = np.ones_like(all_indexes).astype(bool)
test_indexes = all_indexes[test_mask]

X_train, y_train = X[train_indexes, :], y[train_indexes, :]
X_test, y_test = X[test_indexes, :], y[test_indexes, :]


In [4]:
from sklearn.preprocessing import StandardScaler

features_scaler = StandardScaler().fit(X_train)
targets_scaler = StandardScaler().fit(y_train)

X_train_scaled, y_train_scaled = features_scaler.transform(X_train), targets_scaler.transform(y_train)
X_test_scaled, y_test_scaled = features_scaler.transform(X_test), targets_scaler.transform(y_test)

In [5]:
import torch

batch_size = 256

X_train_tensor, y_train_tensor = torch.from_numpy(X_train_scaled).to(torch.float32), torch.from_numpy(y_train_scaled).to(torch.float32)
X_test_tensor, y_test_tensor = torch.from_numpy(X_test_scaled), torch.from_numpy(y_test_scaled)

train_tensor_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_dataloader = torch.utils.data.DataLoader(train_tensor_dataset, batch_size=batch_size)

In [6]:
from pushforward_operators import RectifiedFlowQuantile
from classes import TrainParameters

quantile = RectifiedFlowQuantile(
    feature_dimension=160,
    response_dimension=32,
    hidden_dimension=32,
    number_of_hidden_layers=8,
)

train_parameters = TrainParameters(
    number_of_epochs_to_train=200,
    optimizer_parameters=dict(
        lr=1e-2,
    ),
    scheduler_parameters=dict(
        eta_min=0
    ),
    warmup_iterations=5,
    verbose=True
)

In [7]:
quantile.fit(
    dataloader=train_dataloader,
    train_parameters=train_parameters,
    number_of_rectifying_operations=1,
    jacobian_symmetry_penalty_weight=0.
)

Epoch: 200, Flow Loss: 20.9090, LR: 0.000000: 100%|██████████| 200/200 [07:56<00:00,  2.38s/it]


RectifiedFlowQuantile(
  (time_embedding_network): Sequential(
    (0): Linear(in_features=1, out_features=160, bias=True)
    (1): Tanh()
  )
  (vector_field_network): FFNN(
    (activation_function): Softplus(beta=1.0, threshold=20.0)
    (potential_network): Sequential(
      (0): Linear(in_features=352, out_features=32, bias=True)
      (1): Softplus(beta=1.0, threshold=20.0)
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Softplus(beta=1.0, threshold=20.0)
      (4): Linear(in_features=32, out_features=32, bias=True)
      (5): Softplus(beta=1.0, threshold=20.0)
      (6): Linear(in_features=32, out_features=32, bias=True)
      (7): Softplus(beta=1.0, threshold=20.0)
      (8): Linear(in_features=32, out_features=32, bias=True)
      (9): Softplus(beta=1.0, threshold=20.0)
      (10): Linear(in_features=32, out_features=32, bias=True)
      (11): Softplus(beta=1.0, threshold=20.0)
      (12): Linear(in_features=32, out_features=32, bias=True)
      (13): 

In [8]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


feature_position_x_slice = slice(64, 96)
feature_position_y_slice = slice(96, 128)

target_position_x_slice = slice(0, 16)
target_position_y_slice = slice(16, 32)

for test_idx in range(20):

    number_of_samples = 8
    x_sample, y_sample = X_test_tensor[test_idx: test_idx+1, :], y_test_tensor[test_idx: test_idx+1, :]
    target_space_size = y_sample.shape[-1]
    u_samples = torch.zeros(1, target_space_size)
    x_sample_repeated = x_sample.repeat(1, 1).to(dtype=torch.float32)
    y_scaled = quantile.push_u_given_x(u_samples, x_sample_repeated, number_of_evaluations=1000)
    y_prediction = targets_scaler.inverse_transform(y_scaled.detach().numpy())
    
    x_sample_unscaled = features_scaler.inverse_transform(x_sample.detach().numpy())
    y_sample_unscaled = targets_scaler.inverse_transform(y_sample.detach().numpy())

    map_folderpath = "../attic/data/ATC"
    yaml_path = f"{map_folderpath}/ATC-map/localization_grid.yaml"

    with open(yaml_path, "r") as f:
        meta = yaml.safe_load(f)

    img_path = meta["image"]
    res = float(meta["resolution"])
    origin_x, origin_y, origin_yaw = meta["origin"]

    img = mpimg.imread(f"{map_folderpath}/ATC-map/{img_path}")

    img = np.array(img)
    if img.ndim == 3:
        img = img[..., 0]

    h, w = img.shape

    x0 = origin_x
    y0 = origin_y
    x1 = origin_x + w * res
    y1 = origin_y + h * res

    sample_idx = torch.randint(0, y_prediction.shape[0], (1, ))

    fig, ax = plt.subplots(figsize=(10, 8))

    ax.imshow(img, cmap="gray", extent=[x0, x1, y0, y1])

    ax.set_aspect("equal", adjustable="box")
    ax.set_xlabel("x [m]")
    ax.set_ylabel("y [m]")
    ax.set_title("Scale-accurate building grid")


    ax.scatter(
        x_sample_unscaled[0, feature_position_x_slice] / 1000,
        x_sample_unscaled[0, feature_position_y_slice] / 1000,
        c="blue",
        label="Context"
    )
    ax.scatter(
        y_sample_unscaled[0, target_position_x_slice] / 1000,
        y_sample_unscaled[0, target_position_y_slice] / 1000,
        c="red",
        label="Ground Truth",
        marker='x'
    )

    for sample_idx in range(0, min(8, y_prediction.shape[0])):
        ax.scatter(
            y_prediction[sample_idx, target_position_x_slice] / 1000,
            y_prediction[sample_idx, target_position_y_slice] / 1000,
            s=4,
        )

    ax.legend()
    plt.title(f"Index: {test_idx}")
    plt.show()

AttributeError: 'RectifiedFlowQuantile' object has no attribute '_predict_vector_field'